# Secure Agent Memory — Encryption, Schemas & Auto-Expiry

SafeAI provides a **schema-enforced, encrypted, per-agent memory store** with automatic retention-based expiry.
This notebook walks through every major capability:

1. Writing and reading agent memory
2. Schema enforcement (rejected writes for unknown fields or wrong types)
3. Encrypted fields that return opaque handles instead of raw values
4. Per-agent isolation
5. Auto-expiry driven by field-level retention policies

Memory requires a schema file, so we use `SafeAI.from_config()` with a temporary project directory.

In [1]:
import tempfile
import yaml
from pathlib import Path
from click.testing import CliRunner

from safeai.cli.init import init_command
from safeai import SafeAI

# Create a temporary project directory and scaffold default config files
tmp = tempfile.TemporaryDirectory()
work = Path(tmp.name)
result = CliRunner().invoke(init_command, ["--path", str(work)])
print(result.output)

# Patch the memory schema to add an encrypted field for the demo
schema_path = work / "schemas" / "memory.yaml"
schema = yaml.safe_load(schema_path.read_text())
schema["memory"]["fields"].append({
    "name": "auth_token",
    "type": "string",
    "tag": "confidential",
    "retention": "1h",
    "encrypted": True,
    "required": False,
})
schema_path.write_text(yaml.dump(schema, default_flow_style=False))
print("Patched memory schema — added encrypted field 'auth_token'")

# Build SafeAI from config (this loads the memory schema)
ai = SafeAI.from_config(work / "safeai.yaml")
print(f"Memory controller loaded: {ai.memory is not None}")
print(f"Allowed fields: {ai.memory.allowed_fields}")

SafeAI initialized
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/safeai.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/policies/default.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/contracts/example.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/schemas/memory.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/agents/default.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/plugins/example.py
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/tenants/policy-sets.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpkqc8hqs8/alerts/default.yaml

Patched memory schema — added encrypted field 'auth_token'
Memory controller loaded: True
Allowed fields: {'user_preference', 'auth_token', 'ticket_id'}


## 1. Write and read agent memory

Use `ai.memory_write(key, value, agent_id=...)` to store a value and
`ai.memory_read(key, agent_id=...)` to retrieve it.
The schema defines two plain-text fields (`user_preference` and `ticket_id`)
plus the encrypted `auth_token` we added above.

In [2]:
# Write values as agent-a
ok1 = ai.memory_write("user_preference", "dark-mode", agent_id="agent-a")
ok2 = ai.memory_write("ticket_id", "TICKET-4042", agent_id="agent-a")
print(f"user_preference write: {ok1}")
print(f"ticket_id write:       {ok2}")

# Read them back as agent-a
pref = ai.memory_read("user_preference", agent_id="agent-a")
tid  = ai.memory_read("ticket_id", agent_id="agent-a")
print(f"\nuser_preference read: {pref!r}")
print(f"ticket_id read:       {tid!r}")

# agent-b cannot see agent-a's data
cross = ai.memory_read("user_preference", agent_id="agent-b")
print(f"\nagent-b reads agent-a's user_preference: {cross!r}  (expected None)")

user_preference write: True
ticket_id write:       True

user_preference read: 'dark-mode'
ticket_id read:       'TICKET-4042'

agent-b reads agent-a's user_preference: None  (expected None)


## 2. Schema enforcement — rejected writes

The memory controller only accepts writes to fields declared in the schema,
and the value must match the declared type. Unknown field names or type
mismatches are silently rejected (returns `False`).

In [3]:
# Unknown field — not in the schema
rejected_field = ai.memory_write("secret_plan", "world domination", agent_id="agent-a")
print(f"Write to unknown field 'secret_plan': {rejected_field}  (expected False)")

# Wrong type — ticket_id expects a string, not an integer
rejected_type = ai.memory_write("ticket_id", 9999, agent_id="agent-a")
print(f"Write integer to string field 'ticket_id': {rejected_type}  (expected False)")

# Wrong type — user_preference expects a string, not a list
rejected_list = ai.memory_write("user_preference", ["a", "b"], agent_id="agent-a")
print(f"Write list to string field 'user_preference': {rejected_list}  (expected False)")

# Confirm original values are untouched
print(f"\nuser_preference is still: {ai.memory_read('user_preference', agent_id='agent-a')!r}")
print(f"ticket_id is still:       {ai.memory_read('ticket_id', agent_id='agent-a')!r}")

Write to unknown field 'secret_plan': False  (expected False)
Write integer to string field 'ticket_id': False  (expected False)
Write list to string field 'user_preference': False  (expected False)

user_preference is still: 'dark-mode'
ticket_id is still:       'TICKET-4042'


## 3. Encrypted fields return handles

When a field is marked `encrypted: true` in the schema, the memory controller
encrypts the value at rest using Fernet symmetric encryption. A `memory_read`
returns an opaque **handle ID** (e.g. `hdl_abc123...`) instead of the plaintext
value. To retrieve the actual value, you must call `ai.resolve_memory_handle()`
with the correct `agent_id`.

In [4]:
# Write to the encrypted field
ok = ai.memory_write("auth_token", "sk-live-abc123xyz", agent_id="agent-a")
print(f"auth_token write: {ok}")

# Read returns an opaque handle, not the plaintext
handle = ai.memory_read("auth_token", agent_id="agent-a")
print(f"auth_token read:  {handle!r}")
print(f"Starts with 'hdl_': {str(handle).startswith('hdl_')}")

# Resolve the handle to get the actual value
resolved = ai.resolve_memory_handle(
    handle,
    agent_id="agent-a",
    session_id="sess-001",
    source_agent_id="agent-a",
    destination_agent_id="agent-a",
)
print(f"\nResolved value: {resolved!r}")
print(f"Matches original: {resolved == 'sk-live-abc123xyz'}")

auth_token write: True
auth_token read:  'hdl_18248340fe6649c7838a263d'
Starts with 'hdl_': True

Resolved value: 'sk-live-abc123xyz'
Matches original: True


## 4. Per-agent isolation

Each agent has its own memory namespace. Agent A's data is invisible to Agent B,
and handle resolution is bound to the agent that created the entry.

In [5]:
# agent-a writes a preference
ai.memory_write("user_preference", "compact-view", agent_id="agent-a")

# agent-b writes its own preference
ai.memory_write("user_preference", "expanded-view", agent_id="agent-b")

# Each agent sees only its own value
pref_a = ai.memory_read("user_preference", agent_id="agent-a")
pref_b = ai.memory_read("user_preference", agent_id="agent-b")
print(f"agent-a sees: {pref_a!r}")
print(f"agent-b sees: {pref_b!r}")
print(f"Values are different: {pref_a != pref_b}")

# agent-c has no data at all
pref_c = ai.memory_read("user_preference", agent_id="agent-c")
print(f"\nagent-c sees: {pref_c!r}  (expected None — never wrote anything)")

# Encrypted handle resolution is agent-bound
handle_a = ai.memory_read("auth_token", agent_id="agent-a")
print(f"\nagent-a's auth_token handle: {handle_a!r}")

# agent-b tries to resolve agent-a's handle — fails (returns None)
cross_resolve = ai.resolve_memory_handle(
    handle_a,
    agent_id="agent-b",
    session_id="sess-002",
    source_agent_id="agent-b",
    destination_agent_id="agent-a",
)
print(f"agent-b resolves agent-a's handle: {cross_resolve!r}  (expected None)")

agent-a sees: 'compact-view'
agent-b sees: 'expanded-view'
Values are different: True

agent-c sees: None  (expected None — never wrote anything)

agent-a's auth_token handle: 'hdl_18248340fe6649c7838a263d'
agent-b resolves agent-a's handle: None  (expected None)


## 5. Auto-expiry

Every field in the schema has a **retention** duration (e.g. `30d`, `7d`, `1h`).
Entries are timestamped at write time and expire automatically.

- On every `memory_read` or `memory_write`, expired entries are purged automatically.
- You can also call `ai.memory_purge_expired()` explicitly to sweep all agents.

Since our entries were just created and have long retention windows, the purge
returns `0` here. In production, entries that exceed their retention are silently
removed, and reads return `None`.

In [6]:
# Explicitly purge expired entries across all agents
purged = ai.memory_purge_expired()
print(f"Expired entries purged: {purged}  (expected 0 — entries are fresh)")

# Verify everything is still readable
print(f"\nagent-a user_preference: {ai.memory_read('user_preference', agent_id='agent-a')!r}")
print(f"agent-a ticket_id:       {ai.memory_read('ticket_id', agent_id='agent-a')!r}")
print(f"agent-b user_preference: {ai.memory_read('user_preference', agent_id='agent-b')!r}")

# Show schema-defined retention per field
print("\nField retention settings:")
for field in ai.memory.schema.fields:
    retention = field.retention or ai.memory.schema.default_retention
    print(f"  {field.name:20s}  retention={retention:>4s}  encrypted={field.encrypted}")

Expired entries purged: 0  (expected 0 — entries are fresh)

agent-a user_preference: 'compact-view'
agent-a ticket_id:       'TICKET-4042'
agent-b user_preference: 'expanded-view'

Field retention settings:
  user_preference       retention= 30d  encrypted=False
  ticket_id             retention=  7d  encrypted=False
  auth_token            retention=  1h  encrypted=True


In [7]:
# Cleanup
tmp.cleanup()
print("Temporary directory cleaned up.")

Temporary directory cleaned up.


---

## Summary

SafeAI's memory controller provides a production-ready, schema-enforced store for agent state:

| Capability | How it works |
|---|---|
| **Schema enforcement** | Only declared fields are accepted; type mismatches are rejected |
| **Field-level encryption** | Fields marked `encrypted: true` are Fernet-encrypted at rest; reads return opaque handles |
| **Handle resolution** | `resolve_memory_handle()` decrypts only for the owning agent, with full audit logging |
| **Per-agent isolation** | Each agent has its own namespace; cross-agent reads return `None` |
| **Auto-expiry** | Entries expire based on per-field retention; purged automatically on access or manually via `memory_purge_expired()` |

All operations emit audit events, giving you a complete trail of memory access across your agent fleet.